### Assignment 2 : Prediction Challenge
### Keertana Chidambaram (Collaborator: Dhruval Bhatt)

NOTE: all notes are for Dhruval.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [2]:
df = pd.read_csv('nlsy training set.csv')

In [3]:
df.head()

,E5011701,E5011702,E5011703,E5011704,E5011705,E5011706,E5011707,E5011708,E5011709,E5011710,...,Z9122600,Z9141400,Z9141500,Z9141600,Z9141700,Z9141800,Z9141900,Z9142000,Z9142100,diag.id
0,-4,-4,2,2,2,1,1,2,2,2,...,16,-4,-4,-4,-4,-4,-4,-4,-4,6328
1,-4,-4,-4,-4,-4,-4,-4,2,2,2,...,17,7500,0,0,6,0,7500,0,18,388
2,-4,-4,-4,2,2,4,4,2,2,2,...,16,-4,-4,-4,-4,-4,-4,-4,-4,7655
3,-4,-4,-4,2,2,2,1,1,2,2,...,16,-4,-4,-4,-4,-4,-4,-4,-4,3517
4,-4,-4,-4,-4,-4,-4,2,2,2,2,...,16,-4,-4,-4,-4,-4,-4,-4,-4,8540


In [6]:
df.shape

(7187, 4886)

Unanswered Qs:
- To include or exclude observations with missing data from the test set while building the model??
- If included, should the model predict the 'type' of the missing data?

New learnings:
- Dealing with missing data: treat as another feature, predict feature (mean/median/mode imputation, KNN imputation), skip feature (?)

Notes: 
- Predictor variables are mostly categorical but predicted variable is ordinal categorical
- Error is MSE, so take predicted variable as continuous 
- Missing Ys are not needed, make an error function that accounts for all this
- Since missing Ys not needed, predict missing Y as a -1. That way even if prediction is wrong, we still get small MSE

Stuff we could do:
- One-hot encoding for categorical data
- Regularization
- Remove missing data from train set

List of available models:
1. Regression - NO. used when Xs are continuous
2. Classification - NO. used when predicted variables is categorical
3. Logistic regression - NO. used when predicted variable is binary, multi-class logistic when y is categorical
4. Gradient boosting
5. Decision trees - works!
6. Random forest - works!
7. Neural networks - works!
8. Ensemble methods - works!

In [26]:
sum(df['U1031900'] >= 0) * 100 / df.shape[0]

49.32517044663977

Using ready-made python packages for now

In [ ]:
# Linear regression
# OLS
import statsmodels.api as sm
xvars = ['bin1', 'bin2','bin3','bin4','bin5']
X = data[xvars]
y = data['coolness']
result = sm.OLS(y, X).fit()

# K Nearest Neighbor
from sklearn import neighbors
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=10)
model = neighbors.KNeighborsClassifier(n_neighbors = 2)
x_vars = ['X1', 'X2', 'X3']
train_data[x_vars]
res = model.fit(train_data[x_vars], train_data['Y'])
print('Predicted value for (1,1,1) is:')
print(res.predict([(0,0,0)]))

# Leave one out cross-validation
Xvars = X.values
yvars = y.values
N_loo = Xvars.shape[0]
loo = LeaveOneOut()
loo.get_n_splits(Xvars)
MSE_vec = np.zeros(N_loo)
y_pred = np.zeros(N_loo)

for train_index, test_index in loo.split(Xvars):
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvars[train_index], yvars[test_index]
    LogReg = LogisticRegression(solver='newton-cg',multi_class='multinomial').fit(X_train, y_train)
    y_pred[test_index] = LogReg.predict(X_test)
    
    MSE_vec[test_index] = y_test != y_pred[test_index]

MSE_loo = MSE_vec.mean()
print('test estimate MSE loocv=', MSE_loo)

# K-fold CV
Xvars = X.values
yvars = y.values
k = 4
kf = KFold(n_splits=4, shuffle=True, random_state=10)
kf.get_n_splits(Xvars)

MSE_vec_kf = np.zeros(k)
y_pred = np.zeros(len(yvars))

k_ind = int(0)
for train_index, test_index in kf.split(Xvars):
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvars[train_index], yvars[test_index]
    LogReg = LogisticRegression(solver='newton-cg',multi_class='multinomial').fit(X_train, y_train)
    
    y_pred[test_index] = LogReg.predict(X_test)

    err = y_pred[test_index] != y_test
    MSE_vec_kf[k_ind] = err.mean()
    k_ind += 1

MSE_kf = MSE_vec_kf.mean()
print('test estimate MSE k-fold (k=4) =', MSE_kf)

# Decision trees
xvars = ['female', 'age', 'educ', 'dem', 'rep']
X = data[xvars]
y = data['biden']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=25)
biden_tree = DecisionTreeRegressor(max_depth=3, min_samples_leaf=5)
biden_tree.fit(X_train, y_train)
y_pred = biden_tree.predict(X_test)
MSE1 = mean_squared_error(y_test, y_pred)
print('MSE=', MSE1)

# Search for best tree
param_dist1 = {'max_depth': [3, 10],
               'min_samples_split': sp_randint(2, 20),
               'min_samples_leaf': sp_randint(2, 20)}

biden_tree2 = DecisionTreeRegressor()

random_search1 = RandomizedSearchCV(biden_tree2, 
                                    param_distributions=param_dist1,
                                    n_iter=100, 
                                    n_jobs=-1, 
                                    cv=5, 
                                    random_state=25,
                                    scoring='neg_mean_squared_error')

random_search1.fit(X, y)
print('Best estimator = ', random_search1.best_estimator_)
print('Best tuning parameters = ', random_search1.best_params_)
print('MSE of optimal results =', -random_search1.best_score_)

# Random forest
param_dist2 = { 'n_estimators': [10, 200],
                'max_depth': [3, 10],
                'min_samples_split': sp_randint(2, 20),
                'min_samples_leaf': sp_randint(2, 20),
                'max_features': sp_randint(1, 5)}

biden_tree3 = RandomForestRegressor(bootstrap=True,oob_score=True, random_state=25)

random_search2 = RandomizedSearchCV(biden_tree3, param_distributions=param_dist2, n_iter=100,
                                    n_jobs=-1, cv=5, random_state=25, scoring='neg_mean_squared_error')

random_search2.fit(X, y)
print('Best estimator = ', random_search2.best_estimator_)
print('Best tuning parameters = ', random_search2.best_params_)
print('MSE of optimal results =', -random_search2.best_score_)

# Solution 1.c.
forest = RandomForestClassifier(bootstrap=True, oob_score=True)

param_dist2 = { 'n_estimators': sp_randint(10, 200),
                'max_depth': sp_randint(2, 4),
                'min_samples_split': sp_randint(2, 20),
                'min_samples_leaf': sp_randint(2, 20),
                'max_features': sp_randint(1, 4)}

random_search2 = RandomizedSearchCV(forest, param_distributions=param_dist2, n_iter=200,
                                    n_jobs=-1, cv=5, random_state=25, scoring='neg_mean_squared_error')
random_search2.fit(X, y)
print('Best tuning parameters = ', random_search2.best_params_)
print('MSE of optimal results =', -random_search2.best_score_)

# Solution 1.d.
svc_model = SVC(kernel='rbf')

param_dist3 = { 'C': sp_uniform(loc=0.1, scale=10.0),
                'gamma': ['scale', 'auto'],
                'shrinking': [True, False]}
random_search3 = RandomizedSearchCV(svc_model, param_distributions=param_dist3, n_iter=200,
                                    n_jobs=-1, cv=5, random_state=25, scoring='neg_mean_squared_error')
random_search3.fit(X, y)
print('Best tuning parameters = ', random_search3.best_params_)
print('MSE of optimal results =', -random_search3.best_score_)

# Solution 1.e.
nn = MLPClassifier(solver='lbfgs')

param_dist4 = {'hidden_layer_sizes': sp_randint(1, 100),
                'activation': ['logistic', 'relu'],
                'alpha': sp_uniform(0.1, 10.0)}

random_search4 = RandomizedSearchCV(nn, param_distributions=param_dist4, n_iter=200,
                                    n_jobs=-1, cv=5, random_state=25, scoring='neg_mean_squared_error')
random_search4.fit(X, y)
print('Best tuning parameters = ', random_search4.best_params_)
print('MSE of optimal results =', -random_search4.best_score_)


# K-fold cross validation used to find the average MSE
def K_fold_CV(model, X, y, k = 4):
    Xvals = X.values
    yvals = y.values
    kf = KFold(n_splits=k, shuffle=True, random_state=10)
    kf.get_n_splits(Xvals)
    MSE_vec = np.zeros(k)
    y_pred = np.zeros(len(yvals))

    k_ind = int(0)
    for train_index, test_index in kf.split(Xvals):
        X_train, X_test = Xvals[train_index], Xvals[test_index]
        y_train, y_test = yvals[train_index], yvals[test_index]
        res = model.fit(X_train, y_train)
        y_pred[test_index] = res.predict(X_test)
        err = (y_pred[test_index] - y_test) ** 2
        MSE_vec[k_ind] = err.mean()
        k_ind += 1
    
    return MSE_vec.mean()

# CODES

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import make_scorer
from sklearn.neural_network import MLPClassifier

In [2]:
df = pd.read_csv('../Data/nlsy training set.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.index = df['diag.id']

In [71]:
X = df.loc[:, df.columns != 'U1031900']
y = df['U1031900']

In [73]:
counts = X.where(X>=0).count().sort_values(ascending=False)/X.shape[0] #.sorted()
impute_vars = list(counts.index[:1013])
for col in impute_vars:
    print(col)
    X.loc[X[col]<0, col] = np.nan
    X[col].fillna(list(X[col].mode())[0], inplace=True)
X.head()

,E5011701,E5011702,E5011703,E5011704,E5011705,E5011706,E5011707,E5011708,E5011709,E5011710,...,Z9122600,Z9141400,Z9141500,Z9141600,Z9141700,Z9141800,Z9141900,Z9142000,Z9142100,diag.id
diag.id,,,,,,,,,,,,,,,,,,,,,
6328,-4,-4,2,2,2,1.0,1.0,2.0,2.0,2.0,...,16.0,-4,-4,-4,-4,-4,-4,-4,-4,6328.0
388,-4,-4,-4,-4,-4,2.0,1.0,2.0,2.0,2.0,...,17.0,7500,0,0,6,0,7500,0,18,388.0
7655,-4,-4,-4,2,2,4.0,4.0,2.0,2.0,2.0,...,16.0,-4,-4,-4,-4,-4,-4,-4,-4,7655.0
3517,-4,-4,-4,2,2,2.0,1.0,1.0,2.0,2.0,...,16.0,-4,-4,-4,-4,-4,-4,-4,-4,3517.0
8540,-4,-4,-4,-4,-4,2.0,2.0,2.0,2.0,2.0,...,16.0,-4,-4,-4,-4,-4,-4,-4,-4,8540.0


In [ ]:
# for col in X.columns:
#     new_dummies = pd.get_dummies(X[col])
#     new_dummies.columns = [str(i) + col for i in new_dummies.columns]
#     X = X.join(new_dummies)
# X.head()

In [76]:
def calc_MSE(y_true, y_pred):
    '''Calculates the Mean Squared Error given original and predicted values'''
    y_pred[y_pred<0] = 0
    err = (y_pred[y_true>=0] - y_true[y_true>=0]) ** 2
    return err.mean()
new_scorer = make_scorer(calc_MSE, greater_is_better=False)

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)

In [78]:
model = DecisionTreeRegressor(max_depth=5, min_samples_split=5, min_samples_leaf=40)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(calc_MSE(y_test, y_pred))

20.4368792528934


In [79]:
# Decision Trees
params = {'max_depth': [5, 10, 15],
          'min_samples_split': [2, 5, 10],
          'min_samples_leaf': [30, 40, 50, 60]}
best_MSE = 1000
best_params = {}
for max_depth in params['max_depth']:
    print(max_depth)
    for min_samples_split in params['min_samples_split']:
        for min_samples_leaf in params['min_samples_leaf']:
            model = DecisionTreeRegressor(max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)
            model.fit(X_train,y_train)
            y_pred = model.predict(X_test)
            new_MSE = calc_MSE(y_test, y_pred)
            if new_MSE < best_MSE:
                best_MSE = new_MSE
                best_params['max_depth'] = max_depth
                best_params['min_samples_split'] = min_samples_split
                best_params['min_samples_leaf'] = min_samples_leaf
                best_model = model
print('MSE = ', best_MSE)
print('params = ', best_params)

5
10
15
MSE =  19.887888780513446
params =  {'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 30}


In [26]:
# Random forest regressor
params = {  'n_estimators': [5, 10, 15],
            'max_depth': [7, 10, 15],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [5, 10, 25],
            'max_features': [4000, 4885]}

best_MSE = 1000
best_params = {}
for max_depth in params['max_depth']:
    print(max_depth)
    for min_samples_split in params['min_samples_split']:
        for min_samples_leaf in params['min_samples_leaf']:
            for n_estimators in params['n_estimators']:
                for max_features in params['max_features']:
                    model = RandomForestRegressor(max_depth=max_depth,
                                                  min_samples_split=min_samples_split,
                                                  min_samples_leaf=min_samples_leaf,
                                                  n_estimators=n_estimators, 
                                                  max_features=max_features)
                    model.fit(X_train,y_train)
                    y_pred = model.predict(X_test)
                    new_MSE = calc_MSE(y_test, y_pred)
                    if new_MSE < best_MSE:
                        best_MSE = new_MSE
                        best_params['max_depth'] = max_depth
                        best_params['min_samples_split'] = min_samples_split
                        best_params['min_samples_leaf'] = min_samples_leaf
                        best_params['n_estimators'] = n_estimators
                        best_params['max_features'] = max_features
                        best_model = model
print('MSE = ', best_MSE)
print('params = ', best_params)

7
10
15
MSE =  19.75676459710034
params =  {'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 5, 'n_estimators': 5, 'max_features': 4885}


In [ ]:
# Random forest classifier
params = {  'n_estimators': [5, 10, 15],
            'max_depth': [7, 10, 15],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [5, 10, 25],
            'max_features': [4000, 4885]}

best_MSE = 1000
best_params = {}
for max_depth in params['max_depth']:
    print(max_depth)
    for min_samples_split in params['min_samples_split']:
        for min_samples_leaf in params['min_samples_leaf']:
            for n_estimators in params['n_estimators']:
                for max_features in params['max_features']:
                    model = RandomForestClassifier(max_depth=max_depth,
                                                  min_samples_split=min_samples_split,
                                                  min_samples_leaf=min_samples_leaf,
                                                  n_estimators=n_estimators, 
                                                  max_features=max_features)
                    model.fit(X_train,y_train)
                    y_pred = model.predict(X_test)
                    new_MSE = calc_MSE(y_test, y_pred)
                    if new_MSE < best_MSE:
                        best_MSE = new_MSE
                        best_params['max_depth'] = max_depth
                        best_params['min_samples_split'] = min_samples_split
                        best_params['min_samples_leaf'] = min_samples_leaf
                        best_params['n_estimators'] = n_estimators
                        best_params['max_features'] = max_features
                        best_model = model
print('MSE = ', best_MSE)
print('params = ', best_params)

In [ ]:
# Neural networks
params = { 'hidden_layer_sizes': [5, 10, 15],
            'activation': ['logistic'],
            'alpha': [0.9, 0.7]}
best_MSE = 1000
best_params = {}
for hidden_layer_sizes in params['hidden_layer_sizes']:
    print(hidden_layer_sizes)
    for alpha in params['alpha']:
        model = MLPClassifier(solver='lbfgs', 
                              activation='logistic', 
                              alpha=alpha)
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        new_MSE = calc_MSE(y_test, y_pred)
        if new_MSE < best_MSE:
            best_MSE = new_MSE
            best_params['max_depth'] = max_depth
            best_params['min_samples_split'] = min_samples_split
            best_params['min_samples_leaf'] = min_samples_leaf
            best_model = model
print('MSE = ', best_MSE)
print('params = ', best_params)

In [ ]:
feature_importances = pd.Series(model.feature_importances_, index=X_train.columns)
top_features = list(feature_importances.nlargest(20).index)

In [20]:

param_dist1 = {'max_depth': [3, 5, 10],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [5, 10, 15, 20]}

biden_tree2 = DecisionTreeRegressor()

random_search1 = RandomizedSearchCV(biden_tree2, 
                                    param_distributions=param_dist1,
                                    n_iter=100, 
                                    n_jobs=-1, 
                                    cv=5, 
                                    random_state=25,
                                    scoring=new_scorer)

random_search1.fit(X, y)
print('Best estimator = ', random_search1.best_estimator_)
print('Best tuning parameters = ', random_search1.best_params_)
print('MSE of optimal results =', -random_search1.best_score_)

Best estimator =  DecisionTreeRegressor(criterion='mse', max_depth=3, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=5,
                      min_samples_split=5, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')
Best tuning parameters =  {'min_samples_split': 5, 'min_samples_leaf': 5, 'max_depth': 3}
MSE of optimal results = 22.21571616452844


In [11]:
# Random forest regressor
param_dist2 = { 'n_estimators': [10, 30],
                'max_depth': [3, 5, 7],
                'min_samples_split': [2, 5],
                'min_samples_leaf': [5, 10],
                'max_features': [100, 1000, 4885]}

biden_tree3 = RandomForestRegressor(bootstrap=True,oob_score=True, random_state=25)

random_search2 = RandomizedSearchCV(biden_tree3, param_distributions=param_dist2, n_iter=100,
                                    n_jobs=-1, cv=5, random_state=25, scoring=new_scorer)

random_search2.fit(X, y)
print('Best estimator = ', random_search2.best_estimator_)
print('Best tuning parameters = ', random_search2.best_params_)
print('MSE of optimal results =', -random_search2.best_score_)

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:266: UserWarning: The total space of parameters 72 is smaller than n_iter=100. Running 72 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


Best estimator =  RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=7,
                      max_features=1000, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=10, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=30,
                      n_jobs=None, oob_score=True, random_state=25, verbose=0,
                      warm_start=False)
Best tuning parameters =  {'n_estimators': 30, 'min_samples_split': 2, 'min_samples_leaf': 10, 'max_features': 1000, 'max_depth': 7}
MSE of optimal results = 21.066156751180287


In [12]:
# Random forest classifier
forest = RandomForestClassifier(bootstrap=True, oob_score=True)

param_dist2 = { 'n_estimators': [10, 30],
                'max_depth': [3, 5, 7],
                'min_samples_split': [2, 5],
                'min_samples_leaf': [5, 10],
                'max_features': [100, 1000, 4885]}

random_search2 = RandomizedSearchCV(forest, param_distributions=param_dist2, n_iter=200,
                                    n_jobs=4, cv=5, random_state=25, scoring=new_scorer)
random_search2.fit(X, y)
print('Best estimator = ', random_search2.best_estimator_)
print('Best tuning parameters = ', random_search2.best_params_)
print('MSE of optimal results =', -random_search2.best_score_)

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:266: UserWarning: The total space of parameters 72 is smaller than n_iter=200. Running 72 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
C:\Users\admin\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\admin\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\admin\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means

Best estimator =  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=7, max_features=4885, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)
Best tuning parameters =  {'n_estimators': 10, 'min_samples_split': 5, 'min_samples_leaf': 10, 'max_features': 4885, 'max_depth': 7}
MSE of optimal results = 47.56041624765741


In [ ]:
# Neural networks
nn = MLPClassifier(solver='lbfgs')

param_dist4 = {'hidden_layer_sizes': [8, 10, 12],
                'activation': ['logistic'],
                'alpha': [0.8, 0.7, 0.6]}

random_search4 = RandomizedSearchCV(nn, param_distributions=param_dist4, n_iter=200,
                                    n_jobs=-1, cv=5, random_state=25, scoring=new_scorer)
random_search4.fit(X, y)
print('Best tuning parameters = ', random_search4.best_params_)
print('MSE of optimal results =', -random_search4.best_score_)